# 1. Importing the libraries

In [1401]:
import pandas as pd
import os
import datetime
from pymongo import MongoClient
import pprint

import warnings
warnings.filterwarnings('ignore')

from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
    OrderBy
)

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 7000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

from IPython.display import display, HTML, FileLink
display(HTML('<style>.container { width:100% !important; }</style>'))

# 2. Defining file paths for IRA sessions data

In [1402]:
improve_sessions_path = 'IRA_Sessions_Data/Improve-Sessions-Users.csv'
rebate_sessions_path  = 'IRA_Sessions_Data/Rebate-Sessions-Users.csv'

# 3. Getting the IRA data from MongoDB

In [1403]:
# Define the MongoDB connection string
mongo_url = 'mongodb://**********************************.elb.us-west-1.amazonaws.com:270000/?directConnection=true&authMechanism=SCRAM-SHA-1'

# Create a MongoDB client object
client = MongoClient(
    mongo_url,
    username='***********',
    password='***********',
    authSource='*****'
)

# Access the 'PROD' database
db = client['PROD']

# Access the KenmoreIraEnquiry collection
collection = db['KenmoreIraEnquiry']

# Query the collection with the 'enquiryType': 'enquiry' filter for rebates and 'enquiryType': 'evEnquiry' filter for improves
rebate_results  = collection.find({'enquiryType': 'enquiry'})
improve_results = collection.find({'enquiryType': 'evEnquiry'})

In [1404]:
rebate_results_list = []
for rebate_result in rebate_results:
    rebate_results_list.append(rebate_result)

# Error: IOPub data rate exceeded when trying to print entire rebate_results_list. However, we are able to get all the data in the dataframe.
pprint.pprint(rebate_results_list)

[{'_id': ObjectId('646b17d8c8f62b917c11742e'),
  'address': '111\nsuite',
  'city': 'North Mankato',
  'comments': '',
  'created': datetime.datetime(2023, 4, 4, 11, 12, 23),
  'email': '41kksharma@gmail.com',
  'enquiryType': 'enquiry',
  'firstname': 'Krishna',
  'interestedappliances': ["'Heat Pump Clothes Dryer'",
                           "'Heat Pump Air Conditioner/Heater'",
                           "'Heat Pump Water Heater'"],
  'lastname': 'Sharma',
  'phone': 9008374333,
  'state': '',
  'updated': datetime.datetime(2023, 4, 4, 11, 12, 23),
  'zip': 60604},
 {'_id': ObjectId('646b17d8c8f62b917c11742f'),
  'address': '111 Kings Ct',
  'city': 'North Mankato',
  'comments': '',
  'created': datetime.datetime(2023, 4, 4, 11, 18, 28),
  'email': '41kksharma@gmail.com',
  'enquiryType': 'enquiry',
  'firstname': 'Krishna',
  'interestedappliances': [''],
  'lastname': 'Sharma',
  'phone': 900837433,
  'state': '',
  'updated': datetime.datetime(2023, 4, 4, 11, 18, 28),
  'zip': 

In [1405]:
improve_results_list = []
for improve_result in improve_results:
    improve_results_list.append(improve_result)

pprint.pprint(improve_results_list)

[{'_id': ObjectId('646b17d8c8f62b917c117474'),
  'address': 'D10',
  'city': 'Chicago',
  'comments': '',
  'created': datetime.datetime(2023, 4, 25, 15, 12, 38),
  'email': 'sitansu.jena@transformco.com',
  'enquiryType': 'evEnquiry',
  'firstname': 'sia',
  'interestedappliances': [''],
  'lastname': 'jena',
  'phone': 9299299292,
  'state': 'IL',
  'updated': datetime.datetime(2023, 4, 25, 15, 12, 38),
  'zip': 60169},
 {'_id': ObjectId('646b17d8c8f62b917c117476'),
  'address': 'D10',
  'city': 'Chicago',
  'comments': '',
  'created': datetime.datetime(2023, 4, 25, 16, 1, 9),
  'email': 'sitansu.jena@transformco.com',
  'enquiryType': 'evEnquiry',
  'firstname': 'sia',
  'interestedappliances': [''],
  'lastname': 'jena',
  'phone': 9299299292,
  'state': 'IL',
  'updated': datetime.datetime(2023, 4, 25, 16, 1, 9),
  'zip': 60169},
 {'_id': ObjectId('646b17d8c8f62b917c11747b'),
  'address': '2550 N 1st St',
  'city': 'San Jose',
  'comments': '',
  'created': datetime.datetime(2023

## 3.1 Getting the IRA rebates and improves data into dataframes

In [1406]:
rebate_df = pd.json_normalize(rebate_results_list)
rebate_df.head(3)

,_id,firstname,lastname,phone,email,address,city,zip,interestedappliances,comments,created,enquiryType,state,updated,_class
0,646b17d8c8f62b917c11742e,Krishna,Sharma,9008374333,41kksharma@gmail.com,111\nsuite,North Mankato,60604,"['Heat Pump Clothes Dryer', 'Heat Pump Air Con...",,2023-04-04 11:12:23,enquiry,,2023-04-04 11:12:23,NaN
1,646b17d8c8f62b917c11742f,Krishna,Sharma,900837433,41kksharma@gmail.com,111 Kings Ct,North Mankato,56003,[],,2023-04-04 11:18:28,enquiry,,2023-04-04 11:18:28,NaN
2,646b17d8c8f62b917c117430,Krishna,Sharma,,41kksharma@gmail.com,111,North Mankato,60604,[],,2023-04-04 11:19:32,enquiry,,2023-04-04 11:19:32,NaN


In [1407]:
improve_df = pd.json_normalize(improve_results_list)
improve_df.head(2)

,_id,firstname,lastname,phone,email,address,city,zip,interestedappliances,comments,created,enquiryType,state,updated,_class
0,646b17d8c8f62b917c117474,sia,jena,9299299292,sitansu.jena@transformco.com,D10,Chicago,60169,[],,2023-04-25 15:12:38,evEnquiry,IL,2023-04-25 15:12:38,NaN
1,646b17d8c8f62b917c117476,sia,jena,9299299292,sitansu.jena@transformco.com,D10,Chicago,60169,[],,2023-04-25 16:01:09,evEnquiry,IL,2023-04-25 16:01:09,NaN


## 3.2 Cleaning the IRA rebates dataframe

In [1408]:
rebate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   _id                   72 non-null     object        
 1   firstname             72 non-null     object        
 2   lastname              72 non-null     object        
 3   phone                 72 non-null     object        
 4   email                 72 non-null     object        
 5   address               72 non-null     object        
 6   city                  72 non-null     object        
 7   zip                   72 non-null     object        
 8   interestedappliances  71 non-null     object        
 9   comments              71 non-null     object        
 10  created               72 non-null     datetime64[ns]
 11  enquiryType           72 non-null     object        
 12  state                 67 non-null     object        
 13  updated               

### 3.2.1 Dropping unnecessary columns

In [1409]:
unnecessary_columns = ['_id', 'enquiryType', 'updated']
rebate_df.drop(unnecessary_columns, axis=1, inplace=True)

### 3.2.2 Adding other columns

In [1410]:
rebate_df['Visited Page'] = 'https://rebates.kenmore.com/'
rebate_df['state'] = None

### 3.2.3 Renaming columns

In [1411]:
rebate_df.rename(columns = {
    'firstname': 'First Name',
    'lastname': 'Last Name',
    'phone': 'Phone'
}, inplace = True)

rebate_df.head(2)

,First Name,Last Name,Phone,email,address,city,zip,interestedappliances,comments,created,state,_class,Visited Page
0,Krishna,Sharma,9008374333,41kksharma@gmail.com,111\nsuite,North Mankato,60604,"['Heat Pump Clothes Dryer', 'Heat Pump Air Con...",,2023-04-04 11:12:23,None,NaN,https://rebates.kenmore.com/
1,Krishna,Sharma,900837433,41kksharma@gmail.com,111 Kings Ct,North Mankato,56003,[],,2023-04-04 11:18:28,None,NaN,https://rebates.kenmore.com/


## 3.3 Cleaning the IRA improves dataframe

In [1412]:
improve_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   _id                   17 non-null     object        
 1   firstname             17 non-null     object        
 2   lastname              17 non-null     object        
 3   phone                 17 non-null     object        
 4   email                 17 non-null     object        
 5   address               17 non-null     object        
 6   city                  17 non-null     object        
 7   zip                   17 non-null     object        
 8   interestedappliances  17 non-null     object        
 9   comments              12 non-null     object        
 10  created               17 non-null     datetime64[ns]
 11  enquiryType           17 non-null     object        
 12  state                 17 non-null     object        
 13  updated               

### 3.3.1 Dropping unnecessary columns

In [1413]:
unnecessary_columns = ['_id', 'enquiryType', 'updated']
improve_df.drop(unnecessary_columns, axis=1, inplace=True)

### 3.3.2 Adding other columns

In [1414]:
improve_df['interestedappliances'] = None
improve_df['comments']             = None
improve_df['Visited Page']         = 'https://improve.kenmore.com/'

### 3.3.3 Reordering columns

In [1415]:
ordered_columns = [
    'firstname',
    'lastname',
    'phone',
    'email',
    'address',
    'city',
    'zip',
    'interestedappliances',
    'comments',
    'created',
    'Visited Page',
    'state'
]

improve_df = improve_df[ordered_columns]

### 3.3.4 Renaming columns

In [1416]:
improve_df.rename(columns = {
    'firstname': 'First Name',
    'lastname': 'Last Name',
    'phone': 'Phone'
}, inplace = True)

improve_df.head(2)

,First Name,Last Name,Phone,email,address,city,zip,interestedappliances,comments,created,Visited Page,state
0,sia,jena,9299299292,sitansu.jena@transformco.com,D10,Chicago,60169,None,None,2023-04-25 15:12:38,https://improve.kenmore.com/,IL
1,sia,jena,9299299292,sitansu.jena@transformco.com,D10,Chicago,60169,None,None,2023-04-25 16:01:09,https://improve.kenmore.com/,IL


## 3.4 Appending the IRA rebates and improves dataframes to get the final dataframe

In [1417]:
df = rebate_df.append(improve_df, ignore_index = True)

In [1418]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   First Name            89 non-null     object        
 1   Last Name             89 non-null     object        
 2   Phone                 89 non-null     object        
 3   email                 89 non-null     object        
 4   address               89 non-null     object        
 5   city                  89 non-null     object        
 6   zip                   89 non-null     object        
 7   interestedappliances  71 non-null     object        
 8   comments              71 non-null     object        
 9   created               89 non-null     datetime64[ns]
 10  state                 17 non-null     object        
 11  _class                6 non-null      object        
 12  Visited Page          89 non-null     object        
dtypes: datetime64[ns](1), 

### 3.4.1 Ordering values by created date

In [1419]:
df.sort_values(by='created', inplace=True, ignore_index=True)
df

,First Name,Last Name,Phone,email,address,city,zip,interestedappliances,comments,created,state,_class,Visited Page
0,Krishna,Sharma,9008374333,41kksharma@gmail.com,111\nsuite,North Mankato,60604,"['Heat Pump Clothes Dryer', 'Heat Pump Air Con...",,2023-04-04 11:12:23.000,None,NaN,https://rebates.kenmore.com/
1,Krishna,Sharma,900837433,41kksharma@gmail.com,111 Kings Ct,North Mankato,56003,[],,2023-04-04 11:18:28.000,None,NaN,https://rebates.kenmore.com/
2,Krishna,Sharma,,41kksharma@gmail.com,111,North Mankato,60604,[],,2023-04-04 11:19:32.000,None,NaN,https://rebates.kenmore.com/
3,Shivam,trivedi,,Shiivedi@transformco.com,2550 N 1st St,San Jose,95131,[],,2023-04-04 11:22:41.000,None,NaN,https://rebates.kenmore.com/
4,Krishna,Sharma,,41kksharma@gmail.com,111,Amador City,95601,[],,2023-04-04 12:08:41.000,None,NaN,https://rebates.kenmore.com/
5,caas,dsaad,3423423423,asd@asd.csa,123123,cac,95601,"['Battery Backup', 'Heat Pump Air Conditioner/...",,2023-04-04 12:38:27.000,None,NaN,https://rebates.kenmore.com/
6,dasd,sda,,asd@sd.sd,sada,asd,60089,[],,2023-04-06 17:40:25.000,None,NaN,https://rebates.kenmore.com/
7,sdfs,df,,faf@sda.com,dad,sdasdas,95601,[],,2023-04-06 17:40:57.000,None,NaN,https://rebates.kenmore.com/
8,Pratik,Tamhane,23424322,pratik.tamhane@gmail.com,asfsdfsa,Elk Grove Village,60007,[],,2023-04-07 18:59:23.000,None,NaN,https://rebates.kenmore.com/
9,sad,dd,2342342342,fd@ds.sd,afsad,dfdsf,60089,[],,2023-04-10 06:08:28.000,None,NaN,https://rebates.kenmore.com/


# 4. Getting the current date in mmddyyyy format

In [1420]:
current_time = datetime.datetime.now()
current_date = current_time.strftime('%m%d%Y')
current_date

'07102024'

# 5. Getting the sessions and users data for rebates and improves from GA4
## 5.1 Getting the data

In [1421]:
os.getcwd()

'/home/svairag/Automation/Rebate&Improve'

In [1422]:
def run_report(property_id):
    """Runs a simple report on a Google Analytics 4 property."""

    credential_path = "/home/svairag/Automation/Rebate&Improve/shc-mkt-online-analytics-4fb16bff4aef.json"
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

    # Using a default constructor instructs the client to use the credentials specified in GOOGLE_APPLICATION_CREDENTIALS
    # environment variable.
    client = BetaAnalyticsDataClient()

    request = RunReportRequest(
        property = f'properties/{property_id}',
        dimensions = [Dimension(name='date')],
        metrics = [Metric(name='Sessions'), Metric(name='activeUsers')],
        date_ranges = [DateRange(start_date='30daysAgo', end_date='today')],
        order_bys = [
            OrderBy(
                dimension=OrderBy.DimensionOrderBy(dimension_name='date'),
                desc=True
            )
        ]
    )

    df = pd.DataFrame(columns=['Date', 'Visits', 'Users'])

    response = client.run_report(request)
    for row in response.rows:
        df = df.append(
            {
                'Date'  : row.dimension_values[0].value,
                'Visits': row.metric_values[0].value,
                'Users' : row.metric_values[1].value
            },
            ignore_index=True)

    return df

In [1423]:
if __name__ == '__main__':
    rebates_df = run_report(344393047)  # Property id for rebates data
    print('Rebates DataFrame:')
    print(rebates_df)
    print('\n')

    improves_df = run_report(361838237)  # Property id for improves data
    print('Improves DataFrame:')
    print(improves_df)

Rebates DataFrame:
        Date Visits Users
0   20240708      2     2
1   20240706      1     1
2   20240705      1     1
3   20240704      4     4
4   20240702      6     6
5   20240701      4     4
6   20240630      5     5
7   20240628      6     6
8   20240627      3     3
9   20240626      3     3
10  20240625      7     7
11  20240622      2     2
12  20240621      1     1
13  20240620      1     1
14  20240619      1     1
15  20240618      9     9
16  20240617      1     1
17  20240615      1     1
18  20240614      2     2
19  20240612      2     2
20  20240611      7     7


Improves DataFrame:
       Date Visits Users
0  20240708      1     1
1  20240702      2     2
2  20240701      1     1
3  20240626      1     1
4  20240625      2     2
5  20240618      4     4
6  20240617      2     2
7  20240613      1     1
8  20240612      1     1
9  20240611      2     2


## 5.2 Examining the dataframes

In [1424]:
rebates_df.head(2)

,Date,Visits,Users
0,20240708,2,2
1,20240706,1,1


In [1425]:
rebates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    21 non-null     object
 1   Visits  21 non-null     object
 2   Users   21 non-null     object
dtypes: object(3)
memory usage: 632.0+ bytes


In [1426]:
improves_df.head(2)

,Date,Visits,Users
0,20240708,1,1
1,20240702,2,2


In [1427]:
improves_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    10 non-null     object
 1   Visits  10 non-null     object
 2   Users   10 non-null     object
dtypes: object(3)
memory usage: 368.0+ bytes


## 5.3 Converting the columns to the correct data types

In [1428]:
def change_data_type(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    df[['Visits', 'Users']] = df[['Visits', 'Users']].apply(pd.to_numeric)

In [1429]:
change_data_type(rebates_df)
change_data_type(improves_df)

In [1430]:
rebates_df.head(2)

,Date,Visits,Users
0,2024-07-08,2,2
1,2024-07-06,1,1


In [1431]:
rebates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    21 non-null     object
 1   Visits  21 non-null     int64 
 2   Users   21 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 632.0+ bytes


In [1432]:
improves_df.head(2)

,Date,Visits,Users
0,2024-07-08,1,1
1,2024-07-02,2,2


In [1433]:
improves_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    10 non-null     object
 1   Visits  10 non-null     int64 
 2   Users   10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


## 5.4 Writing the dfs to csv files

In [1434]:
rebates_df.to_csv(rebate_sessions_path, index = False)
improves_df.to_csv(improve_sessions_path, index = False)

# 6. Saving the final dataframes to excel

In [1435]:
# Replace file name with latest date
file_name = 'IRA_Rebates_Data_' + current_date + '.xlsx'

with pd.ExcelWriter(file_name, engine = 'xlsxwriter') as writer:
    # Convert the dataframe to XlsxWriter Excel objects
    df.to_excel(writer, sheet_name=f'IRA_Rebates_Data_{current_date}', index=False)
    improves_df.to_excel(writer, sheet_name = 'Improve_Sessions_Users', index = False)
    rebates_df.to_excel(writer, sheet_name = 'Rebate_Sessions_Users', index = False)
    
    # Get the xlsxwriter objects from the dataframe writer objects
    workbook                   = writer.book
    ira_improve_data_worksheet = writer.sheets[f'IRA_Rebates_Data_{current_date}']
    improves_worksheet         = writer.sheets['Improve_Sessions_Users']
    rebates_worksheet          = writer.sheets['Rebate_Sessions_Users']
    
    # Adjusting column widths
    ira_improve_data_worksheet.set_column(0, 2, 15)
    ira_improve_data_worksheet.set_column(3, 3, 30)
    ira_improve_data_worksheet.set_column(4, 4, 20)
    ira_improve_data_worksheet.set_column(5, 5, 15)
    ira_improve_data_worksheet.set_column(7, 7, 45)
    ira_improve_data_worksheet.set_column(8, 8, 15)
    ira_improve_data_worksheet.set_column(9, 9, 20)
    ira_improve_data_worksheet.set_column(11, 11, 30)
    
    improves_worksheet.set_column(0, 2, 13)
    rebates_worksheet.set_column(0, 2, 13)

    writer.save()
    workbook.close()
    
FileLink(file_name)

/home/svairag/Automation/Rebate&Improve/IRA_Rebates_Data_07102024.xlsx